# Обработка датасетов с различными методами кодирования
Этот ноутбук обрабатывает датасеты из `datasets/original/` используя три метода кодирования:
- One-Hot Encoding
- Label Encoding
- Frequency Encoding

In [212]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import os
import ast
from pathlib import Path

## 1. Загрузка реестра датасетов

In [213]:
def load_datasets_registry(registry_path='../datasets/datasets_registry.csv'):
    """
    Загружает реестр датасетов и преобразует его в список структур.

    Args:
        registry_path: путь к файлу datasets_registry.csv

    Returns:
        list: список словарей с информацией о датасетах
    """
    # Читаем CSV с учетом многострочных значений
    df = pd.read_csv(registry_path, skipinitialspace=True)

    datasets_list = []
    for _, row in df.iterrows():
        # Проверяем, является ли значение в cat_col пустым (NaN)
        if pd.isna(row['cat_col']):
            cat_cols_list = []
        else:
            # Обрабатываем cat_col - он может быть многострочным
            cat_col_str = str(row['cat_col']).replace('\n', '').replace('\r', '').strip()
            try:
                # Проверяем, не пустая ли строка после очистки
                if cat_col_str:
                    cat_cols_list = ast.literal_eval(cat_col_str)
                else:
                    cat_cols_list = []
            except (ValueError, SyntaxError):
                # Если строка все равно не парсится, считаем ее пустой
                cat_cols_list = []

        dataset_info = {
            'dataset_name': row['dataset_name'].strip(),
            'dataset_path': row['dataset_path'].strip(),
            'dataset_csv': row['dataset_csv'].strip(),
            'target': row['target'].strip(),
            'cat_cols': cat_cols_list
        }
        datasets_list.append(dataset_info)

    return datasets_list


In [214]:
# Загружаем реестр датасетов
# Указываем корректный путь к файлу относительно расположения ноутбука
datasets = load_datasets_registry(registry_path='../datasets/datasets_registry.csv')
print(f"Загружено {len(datasets)} датасет(ов)")
print("\nПример структуры:")
print(datasets[0])


Загружено 2 датасет(ов)

Пример структуры:
{'dataset_name': 'adult', 'dataset_path': 'datasets/original/adult.csv', 'dataset_csv': 'datasets/adult/data.csv', 'target': 'class', 'cat_cols': ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'class']}


## 2. Функция One-Hot Encoding

In [215]:
def ohe_encoding(datasets_list):
    """
    Применяет One-Hot Encoding к категориальным признакам датасетов.
    Сохраняет результат в папку датасета и добавляет запись в data.csv.

    Args:
        datasets_list: список словарей с информацией о датасетах
    """
    for dataset_info in datasets_list:
        try:
            print(f"\nОбработка {dataset_info['dataset_name']} - One-Hot Encoding...")

            # Загружаем датасет, корректируя путь
            original_path = Path(dataset_info['dataset_path'])
            df = pd.read_csv(Path('..') / original_path)
            print(f"  Загружено {len(df)} строк, {len(df.columns)} колонок")

            # Получаем категориальные колонки (исключая target, если он в списке)
            cat_cols = [col for col in dataset_info['cat_cols'] if col in df.columns]
            target = dataset_info['target']

            # Удаляем target из cat_cols если он там есть
            cat_cols_to_encode = [col for col in cat_cols if col != target]

            # Применяем One-Hot Encoding с типом int
            df_encoded = pd.get_dummies(df, columns=cat_cols_to_encode, prefix_sep='_', drop_first=False, dtype=int)

            # Всегда применяем Label Encoding к целевой колонке
            if target in df_encoded.columns:
                le = LabelEncoder()
                df_encoded[target] = le.fit_transform(df_encoded[target].astype(str))

            # Получаем список новых категориальных колонок
            new_cat_cols = [col for col in df_encoded.columns if col not in df.columns or col in cat_cols_to_encode]
            # Добавляем target к новым колонкам если он категориальный
            if target in dataset_info['cat_cols']:
                new_cat_cols.append(target)

            # Создаем папку для датасета, корректируя путь
            dataset_folder = Path('..') / Path(dataset_info['dataset_csv']).parent
            dataset_folder.mkdir(parents=True, exist_ok=True)

            # Сохраняем закодированный датасет
            output_path = dataset_folder / f"{dataset_info['dataset_name']}_ohe.csv"
            df_encoded.to_csv(output_path, index=False)
            print(f"  Сохранено в {output_path}")
            print(f"  Новый размер: {len(df_encoded)} строк, {len(df_encoded.columns)} колонок")

            # Обновляем запись в data.csv
            data_csv_path = Path('..') / dataset_info['dataset_csv']
            method_name = 'one_hot_encoding'

            if os.path.exists(data_csv_path):
                data_df = pd.read_csv(data_csv_path)
                # Удаляем старую запись для этого метода, если она существует
                data_df = data_df[data_df['method'] != method_name]
            else:
                data_df = pd.DataFrame(columns=['method', 'path', 'New_cat_cols', 'model_path', 'schedul_path',
                                                 'JS divergence', 'Utility_XGBoost', 'Utility_logistic_r', 'matrix_distance'])

            # Добавляем новую запись
            new_row = {
                'method': method_name,
                'path': str(output_path.relative_to(Path('..'))), # Сохраняем относительный путь
                'New_cat_cols': str(new_cat_cols),
                'model_path': '',
                'schedul_path': '',
                'JS divergence': '',
                'Utility_XGBoost': '',
                'Utility_logistic_r': '',
                'matrix_distance': ''
            }
            data_df = pd.concat([data_df, pd.DataFrame([new_row])], ignore_index=True)
            data_df.to_csv(data_csv_path, index=False)
            print(f"  Запись для '{method_name}' обновлена в {data_csv_path}")

        except Exception as e:
            print(f"  Ошибка при обработке {dataset_info['dataset_name']}: {e}")


## 3. Функция Label Encoding

In [216]:
def label_encoding(datasets_list):
    """
    Применяет Label Encoding к категориальным признакам датасетов.
    Сохраняет результат в папку датасета и добавляет запись в data.csv.

    Args:
        datasets_list: список словарей с информацией о датасетах
    """
    for dataset_info in datasets_list:
        try:
            print(f"\nОбработка {dataset_info['dataset_name']} - Label Encoding...")

            # Загружаем датасет, корректируя путь
            original_path = Path(dataset_info['dataset_path'])
            df = pd.read_csv(Path('..') / original_path)
            print(f"  Загружено {len(df)} строк, {len(df.columns)} колонок")

            # Получаем категориальные колонки
            cat_cols = [col for col in dataset_info['cat_cols'] if col in df.columns]
            target = dataset_info['target']

            # Применяем Label Encoding
            df_encoded = df.copy()
            encoded_cols = []

            for col in cat_cols:
                le = LabelEncoder()
                df_encoded[col] = le.fit_transform(df[col].astype(str))
                encoded_cols.append(col)

            # Всегда применяем Label Encoding к целевой колонке (если она еще не была обработана)
            if target in df_encoded.columns and target not in encoded_cols:
                le = LabelEncoder()
                df_encoded[target] = le.fit_transform(df_encoded[target].astype(str))
                encoded_cols.append(target)

            # Формируем список новых категориальных колонок (все закодированные колонки)
            new_cat_cols = encoded_cols

            # Создаем папку для датасета, корректируя путь
            dataset_folder = Path('..') / Path(dataset_info['dataset_csv']).parent
            dataset_folder.mkdir(parents=True, exist_ok=True)

            # Сохраняем закодированный датасет
            output_path = dataset_folder / f"{dataset_info['dataset_name']}_label.csv"
            df_encoded.to_csv(output_path, index=False)
            print(f"  Сохранено в {output_path}")
            print(f"  Размер: {len(df_encoded)} строк, {len(df_encoded.columns)} колонок")
            print(f"  Закодированные колонки: {new_cat_cols}")

            # Обновляем запись в data.csv
            data_csv_path = Path('..') / dataset_info['dataset_csv']
            method_name = 'label_encoding'

            if os.path.exists(data_csv_path):
                data_df = pd.read_csv(data_csv_path)
                # Удаляем старую запись для этого метода, если она существует
                data_df = data_df[data_df['method'] != method_name]
            else:
                data_df = pd.DataFrame(columns=['method', 'path', 'New_cat_cols', 'model_path', 'schedul_path',
                                                 'JS divergence', 'Utility_XGBoost', 'Utility_logistic_r', 'matrix_distance'])

            # Добавляем новую запись
            new_row = {
                'method': method_name,
                'path': str(output_path.relative_to(Path('..'))), # Сохраняем относительный путь
                'New_cat_cols': str(new_cat_cols),
                'model_path': '',
                'schedul_path': '',
                'JS divergence': '',
                'Utility_XGBoost': '',
                'Utility_logistic_r': '',
                'matrix_distance': ''
            }
            data_df = pd.concat([data_df, pd.DataFrame([new_row])], ignore_index=True)
            data_df.to_csv(data_csv_path, index=False)
            print(f"  Запись для '{method_name}' обновлена в {data_csv_path}")

        except Exception as e:
            print(f"  Ошибка при обработке {dataset_info['dataset_name']}: {e}")

## 4. Функция Frequency Encoding

In [217]:
def frequency_encoding(datasets_list):
    """
    Применяет Frequency Encoding к категориальным признакам датасетов.
    Сохраняет результат в папку датасета и добавляет запись в data.csv.

    Args:
        datasets_list: список словарей с информацией о датасетах
    """
    for dataset_info in datasets_list:
        try:
            print(f"\nОбработка {dataset_info['dataset_name']} - Frequency Encoding...")

            # Загружаем датасет, корректируя путь
            original_path = Path(dataset_info['dataset_path'])
            df = pd.read_csv(Path('..') / original_path)
            print(f"  Загружено {len(df)} строк, {len(df.columns)} колонок")

            # Получаем категориальные колонки
            cat_cols = [col for col in dataset_info['cat_cols'] if col in df.columns]
            target = dataset_info['target']

            # Применяем Frequency Encoding (но не к target)
            df_encoded = df.copy()
            encoded_cols = []

            for col in cat_cols:
                # Пропускаем target, он будет обработан отдельно через LabelEncoder
                if col == target:
                    continue
                # Вычисляем частоты
                freq_map = df[col].value_counts(normalize=True).to_dict()
                # Заменяем значения на частоты
                df_encoded[col] = df[col].map(freq_map)
                encoded_cols.append(col)

            # Всегда применяем Label Encoding к целевой колонке
            if target in df_encoded.columns:
                le = LabelEncoder()
                df_encoded[target] = le.fit_transform(df_encoded[target].astype(str))
                encoded_cols.append(target)

            # Формируем список новых категориальных колонок (все закодированные колонки)
            new_cat_cols = encoded_cols

            # Создаем папку для датасета, корректируя путь
            dataset_folder = Path('..') / Path(dataset_info['dataset_csv']).parent
            dataset_folder.mkdir(parents=True, exist_ok=True)

            # Сохраняем закодированный датасет
            output_path = dataset_folder / f"{dataset_info['dataset_name']}_frequency.csv"
            df_encoded.to_csv(output_path, index=False)
            print(f"  Сохранено в {output_path}")
            print(f"  Размер: {len(df_encoded)} строк, {len(df_encoded.columns)} колонок")
            print(f"  Закодированные колонки: {new_cat_cols}")

            # Обновляем запись в data.csv
            data_csv_path = Path('..') / dataset_info['dataset_csv']
            method_name = 'frequency_encoding'

            if os.path.exists(data_csv_path):
                data_df = pd.read_csv(data_csv_path)
                # Удаляем старую запись для этого метода, если она существует
                data_df = data_df[data_df['method'] != method_name]
            else:
                data_df = pd.DataFrame(columns=['method', 'path', 'New_cat_cols', 'model_path', 'schedul_path',
                                                 'JS divergence', 'Utility_XGBoost', 'Utility_logistic_r', 'matrix_distance'])

            # Добавляем новую запись
            new_row = {
                'method': method_name,
                'path': str(output_path.relative_to(Path('..'))), # Сохраняем относительный путь
                'New_cat_cols': str(new_cat_cols),
                'model_path': '',
                'schedul_path': '',
                'JS divergence': '',
                'Utility_XGBoost': '',
                'Utility_logistic_r': '',
                'matrix_distance': ''
            }
            data_df = pd.concat([data_df, pd.DataFrame([new_row])], ignore_index=True)
            data_df.to_csv(data_csv_path, index=False)
            print(f"  Запись для '{method_name}' обновлена в {data_csv_path}")

        except Exception as e:
            print(f"  Ошибка при обработке {dataset_info['dataset_name']}: {e}")

## 5. Функция Original (без кодирования, только target)

In [218]:
def original_encoding(datasets_list):
    """
    Сохраняет оригинальный датасет с Label Encoding только для целевой колонки.
    Все остальные колонки остаются без изменений.

    Args:
        datasets_list: список словарей с информацией о датасетах
    """
    for dataset_info in datasets_list:
        try:
            print(f"\nОбработка {dataset_info['dataset_name']} - Original (только target закодирован)...")

            # Загружаем датасет, корректируя путь
            original_path = Path(dataset_info['dataset_path'])
            df = pd.read_csv(Path('..') / original_path)
            print(f"  Загружено {len(df)} строк, {len(df.columns)} колонок")

            target = dataset_info['target']

            # Копируем датасет без изменений
            df_encoded = df.copy()

            # Применяем Label Encoding только к целевой колонке
            if target in df_encoded.columns:
                le = LabelEncoder()
                df_encoded[target] = le.fit_transform(df_encoded[target].astype(str))
                print(f"  Целевая колонка '{target}' закодирована с помощью LabelEncoder")

            # Список категориальных колонок - только target (если он был категориальным)
            new_cat_cols = [target] if target in dataset_info['cat_cols'] and target in df_encoded.columns else []

            # Создаем папку для датасета, корректируя путь
            dataset_folder = Path('..') / Path(dataset_info['dataset_csv']).parent
            dataset_folder.mkdir(parents=True, exist_ok=True)

            # Сохраняем датасет
            output_path = dataset_folder / f"{dataset_info['dataset_name']}_original.csv"
            df_encoded.to_csv(output_path, index=False)
            print(f"  Сохранено в {output_path}")
            print(f"  Размер: {len(df_encoded)} строк, {len(df_encoded.columns)} колонок")
            print(f"  Закодированные колонки: {new_cat_cols}")

            # Обновляем запись в data.csv
            data_csv_path = Path('..') / dataset_info['dataset_csv']
            method_name = 'original'

            if os.path.exists(data_csv_path):
                data_df = pd.read_csv(data_csv_path)
                # Удаляем старую запись для этого метода, если она существует
                data_df = data_df[data_df['method'] != method_name]
            else:
                data_df = pd.DataFrame(columns=['method', 'path', 'New_cat_cols', 'model_path', 'schedul_path',
                                                 'JS divergence', 'Utility_XGBoost', 'Utility_logistic_r', 'matrix_distance'])

            # Добавляем новую запись
            new_row = {
                'method': method_name,
                'path': str(output_path.relative_to(Path('..'))), # Сохраняем относительный путь
                'New_cat_cols': str(new_cat_cols),
                'model_path': '',
                'schedul_path': '',
                'JS divergence': '',
                'Utility_XGBoost': '',
                'Utility_logistic_r': '',
                'matrix_distance': ''
            }
            data_df = pd.concat([data_df, pd.DataFrame([new_row])], ignore_index=True)
            data_df.to_csv(data_csv_path, index=False)
            print(f"  Запись для '{method_name}' обновлена в {data_csv_path}")

        except Exception as e:
            print(f"  Ошибка при обработке {dataset_info['dataset_name']}: {e}")

## 6. Запуск обработки датасетов

In [219]:
# Применяем One-Hot Encoding
print("="*50)
print("ONE-HOT ENCODING")
print("="*50)
ohe_encoding(datasets)

ONE-HOT ENCODING

Обработка adult - One-Hot Encoding...
  Загружено 10000 строк, 15 колонок
  Сохранено в ../datasets/adult/adult_ohe.csv
  Новый размер: 10000 строк, 104 колонок
  Запись для 'one_hot_encoding' обновлена в ../datasets/adult/data.csv

Обработка magic_gamma - One-Hot Encoding...
  Загружено 10000 строк, 11 колонок
  Сохранено в ../datasets/magic_gamma/magic_gamma_ohe.csv
  Новый размер: 10000 строк, 11 колонок
  Запись для 'one_hot_encoding' обновлена в ../datasets/magic_gamma/data.csv


In [220]:
# Применяем Label Encoding
print("\n" + "="*50)
print("LABEL ENCODING")
print("="*50)
label_encoding(datasets)


LABEL ENCODING

Обработка adult - Label Encoding...
  Загружено 10000 строк, 15 колонок
  Сохранено в ../datasets/adult/adult_label.csv
  Размер: 10000 строк, 15 колонок
  Закодированные колонки: ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'class']
  Запись для 'label_encoding' обновлена в ../datasets/adult/data.csv

Обработка magic_gamma - Label Encoding...
  Загружено 10000 строк, 11 колонок
  Сохранено в ../datasets/magic_gamma/magic_gamma_label.csv
  Размер: 10000 строк, 11 колонок
  Закодированные колонки: ['class:']
  Запись для 'label_encoding' обновлена в ../datasets/magic_gamma/data.csv


In [221]:
# Применяем Frequency Encoding
print("\n" + "="*50)
print("FREQUENCY ENCODING")
print("="*50)
frequency_encoding(datasets)


FREQUENCY ENCODING

Обработка adult - Frequency Encoding...
  Загружено 10000 строк, 15 колонок
  Сохранено в ../datasets/adult/adult_frequency.csv
  Размер: 10000 строк, 15 колонок
  Закодированные колонки: ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'class']
  Запись для 'frequency_encoding' обновлена в ../datasets/adult/data.csv

Обработка magic_gamma - Frequency Encoding...
  Загружено 10000 строк, 11 колонок
  Сохранено в ../datasets/magic_gamma/magic_gamma_frequency.csv
  Размер: 10000 строк, 11 колонок
  Закодированные колонки: ['class:']
  Запись для 'frequency_encoding' обновлена в ../datasets/magic_gamma/data.csv


In [222]:
# Применяем Original Encoding
print("\n" + "="*50)
print("ORIGINAL ENCODING")
print("="*50)
original_encoding(datasets)


ORIGINAL ENCODING

Обработка adult - Original (только target закодирован)...
  Загружено 10000 строк, 15 колонок
  Целевая колонка 'class' закодирована с помощью LabelEncoder
  Сохранено в ../datasets/adult/adult_original.csv
  Размер: 10000 строк, 15 колонок
  Закодированные колонки: ['class']
  Запись для 'original' обновлена в ../datasets/adult/data.csv

Обработка magic_gamma - Original (только target закодирован)...
  Загружено 10000 строк, 11 колонок
  Целевая колонка 'class:' закодирована с помощью LabelEncoder
  Сохранено в ../datasets/magic_gamma/magic_gamma_original.csv
  Размер: 10000 строк, 11 колонок
  Закодированные колонки: ['class:']
  Запись для 'original' обновлена в ../datasets/magic_gamma/data.csv


## 7. Проверка результатов

In [223]:
# Просмотр обновленного data.csv для датасета adult
data_csv_path = '../datasets/adult/data.csv'
if os.path.exists(data_csv_path):
    result_df = pd.read_csv(data_csv_path)
    print(f"Содержимое {data_csv_path}:")
    result_df.head()
else:
    print(f"Файл {data_csv_path} не найден")

Содержимое ../datasets/adult/data.csv:


In [224]:
result_df

,method,path,New_cat_cols,model_path,schedul_path,JS divergence,Utility_XGBoost,Utility_logistic_r,matrix_distance
0,one_hot_encoding,datasets/adult/adult_ohe.csv,"['workclass_Federal-gov', 'workclass_Local-gov...",NaN,NaN,NaN,NaN,NaN,NaN
1,label_encoding,datasets/adult/adult_label.csv,"['workclass', 'education', 'marital-status', '...",NaN,NaN,NaN,NaN,NaN,NaN
2,frequency_encoding,datasets/adult/adult_frequency.csv,"['workclass', 'education', 'marital-status', '...",NaN,NaN,NaN,NaN,NaN,NaN
3,original,datasets/adult/adult_original.csv,['class'],NaN,NaN,NaN,NaN,NaN,NaN


In [225]:
# Сравнение размеров датасетов
print("\nСравнение размеров датасетов:")
print("-" * 80)

for dataset_info in datasets:
    dataset_folder = Path(dataset_info['dataset_csv']).parent
    dataset_name = dataset_info['dataset_name']
    
    print(f"\n{dataset_name.upper()}:")
    
    # Оригинальный датасет
    if os.path.exists(dataset_info['dataset_path']):
        df_orig = pd.read_csv(dataset_info['dataset_path'])
        print(f"  Оригинал:    {df_orig.shape[0]:>6} строк × {df_orig.shape[1]:>3} колонок")
    
    # OHE
    ohe_path = dataset_folder / f"{dataset_name}_ohe.csv"
    if os.path.exists(ohe_path):
        df_ohe = pd.read_csv(ohe_path)
        print(f"  OHE:         {df_ohe.shape[0]:>6} строк × {df_ohe.shape[1]:>3} колонок")
    
    # Label
    label_path = dataset_folder / f"{dataset_name}_label.csv"
    if os.path.exists(label_path):
        df_label = pd.read_csv(label_path)
        print(f"  Label:       {df_label.shape[0]:>6} строк × {df_label.shape[1]:>3} колонок")
    
    # Frequency
    freq_path = dataset_folder / f"{dataset_name}_frequency.csv"
    if os.path.exists(freq_path):
        df_freq = pd.read_csv(freq_path)
        print(f"  Frequency:   {df_freq.shape[0]:>6} строк × {df_freq.shape[1]:>3} колонок")

    # Original
    orig_path = dataset_folder / f"{dataset_name}_original.csv"
    if os.path.exists(orig_path):
        df_orig_enc = pd.read_csv(orig_path)
        print(f"  Original:    {df_orig_enc.shape[0]:>6} строк × {df_orig_enc.shape[1]:>3} колонок")



Сравнение размеров датасетов:
--------------------------------------------------------------------------------

ADULT:

MAGIC_GAMMA:
